# Contexto analítico

Este caso se basa en un reto propuesto por Red Hat en la plataforma [Kaggle](https://www.kaggle.com/c/predicting-red-hat-business-value/data). 

Red Hat es capaz de obtener una importante cantidad de información de comportamiento de sus clientes. El interes de la compañia se basa en la busqueda de mejores metodos para analizar sus clientes para predecir sobre cuales deberian hacer un especial enfasis que podrian convertirse en clientes de alto valor.

En este reto se propone crear un algoritmo de clasificación que identifique los clientes con mayor potencial para convertirse en clientes de alto valor para Red Hat basados en su actividad.

Se cuenta con dos datasets (`people.csv` y `act_train.csv`) los cuales se cargaran directamente a Colab una vez comprimidos.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/My Drive/Colab Notebooks/Platzi/curso-redes-neuronales/github/clasificacion'
%ls 

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/Platzi/curso-redes-neuronales/github/clasificacion
datasets/  ingenieria_de_datos.ipynb


In [29]:
#Importamos las librerias necesarias
import pandas as pd
import numpy as np

people = pd.read_csv('datasets/people.zip', sep=',')
people.head(3)

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,char_11,char_12,char_13,char_14,char_15,char_16,char_17,char_18,char_19,char_20,char_21,char_22,char_23,char_24,char_25,char_26,char_27,char_28,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,type 2,type 2,True,False,False,True,True,False,True,False,False,False,False,True,False,False,False,False,False,True,True,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,type 2,type 4,False,False,True,True,False,False,False,True,False,False,False,False,False,True,False,True,True,True,False,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,type 2,type 2,True,True,True,True,True,True,False,True,False,True,False,True,True,True,True,True,True,True,True,False,False,True,True,True,True,False,True,True,99


El primer hecho a resaltar es que la información se presenta anonimizada, algo muy común en la ciencia de datos. 

Se aprecia una gran cantidad de campos de naturaleza categórica. 



In [30]:
activity = pd.read_csv('datasets/act_train.zip', sep=',')

activity.head(3)

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


Se puede notar que existe la misma llave en la columna `people_id`, por lo que se puede empezar a pensar en algún tipo de cruce entre estos datos.

# <h1 id="ingenieria">Ingeniería de datos</h1>

Evaluaremos ahora el tamaño de los datasets y la completitud de información en las distintas variables disponibles.

## Eliminando datos nulos

In [4]:
print(people.shape)
100*people.isnull().sum()/people.shape[0]

(189118, 41)


people_id    0.0
char_1       0.0
group_1      0.0
char_2       0.0
date         0.0
char_3       0.0
char_4       0.0
char_5       0.0
char_6       0.0
char_7       0.0
char_8       0.0
char_9       0.0
char_10      0.0
char_11      0.0
char_12      0.0
char_13      0.0
char_14      0.0
char_15      0.0
char_16      0.0
char_17      0.0
char_18      0.0
char_19      0.0
char_20      0.0
char_21      0.0
char_22      0.0
char_23      0.0
char_24      0.0
char_25      0.0
char_26      0.0
char_27      0.0
char_28      0.0
char_29      0.0
char_30      0.0
char_31      0.0
char_32      0.0
char_33      0.0
char_34      0.0
char_35      0.0
char_36      0.0
char_37      0.0
char_38      0.0
dtype: float64

In [5]:
#Repetimos el procedimiento con el dataset de activity
print(activity.shape)
100*activity.isnull().sum()/activity.shape[0]

(2197291, 15)


people_id             0.000000
activity_id           0.000000
date                  0.000000
activity_category     0.000000
char_1               92.826849
char_2               92.826849
char_3               92.826849
char_4               92.826849
char_5               92.826849
char_6               92.826849
char_7               92.826849
char_8               92.826849
char_9               92.826849
char_10               7.173151
outcome               0.000000
dtype: float64

Se evidencia que los campos `char_1` al `char_9 ` tienen el 92% de sus valores en nulo Esta cantidad es muy elevada para aplicar metodos de imputación, por lo cual optaremos por elminar las columnas con este inconveniente.

En el caso de la variable categórica `char_10` existe una proporción de nulos (7.17\%) que podemos reemplazar por la moda.

In [6]:
activity.columns

Index(['people_id', 'activity_id', 'date', 'activity_category', 'char_1',
       'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7', 'char_8',
       'char_9', 'char_10', 'outcome'],
      dtype='object')

In [7]:
# eliminar columnas con valores nulos
activity.drop(columns=['char_1','char_2', 'char_3', 'char_4', 'char_5', 'char_6',
                       'char_7', 'char_8','char_9'],inplace=True)

# cambiar valores nulos por la moda
activity['char_10'].fillna(activity['char_10'].mode()[0], inplace=True)

#Verificamos el cambio
100*activity.isnull().sum()/activity.shape[0]

people_id            0.0
activity_id          0.0
date                 0.0
activity_category    0.0
char_10              0.0
outcome              0.0
dtype: float64

## Preparando para el merge




Se aprecia que existen dos variables que tienen los mismos nombres en ambos datasets (`date` y `char_10`) por lo cual es buena practica renombrar estas columnas para evitar confusiones al momento de unificar las fuentes. 




In [8]:
#Renombramos columnas
activity = activity.rename(columns={"date":"fecha_actividad","char_10":"tipo_actividad"})

In [9]:
# verificamos el cambio
print(activity.columns)
print(people.columns)

Index(['people_id', 'activity_id', 'fecha_actividad', 'activity_category',
       'tipo_actividad', 'outcome'],
      dtype='object')
Index(['people_id', 'char_1', 'group_1', 'char_2', 'date', 'char_3', 'char_4',
       'char_5', 'char_6', 'char_7', 'char_8', 'char_9', 'char_10', 'char_11',
       'char_12', 'char_13', 'char_14', 'char_15', 'char_16', 'char_17',
       'char_18', 'char_19', 'char_20', 'char_21', 'char_22', 'char_23',
       'char_24', 'char_25', 'char_26', 'char_27', 'char_28', 'char_29',
       'char_30', 'char_31', 'char_32', 'char_33', 'char_34', 'char_35',
       'char_36', 'char_37', 'char_38'],
      dtype='object')


Ahora unificamos los datasets empleando la función `merge` y evaluamos la consistencia luego de esta operación






In [10]:
consolidado = activity.merge(people,on=["people_id"],how="inner")
print("Activity tamaño previo: ",activity.shape)
print("People tamaño previo: ",people.shape)
print()
print("Nuevo tamaño: ",consolidado.shape)

Activity tamaño previo:  (2197291, 6)
People tamaño previo:  (189118, 41)

Nuevo tamaño:  (2197291, 46)


## Transformación de datos

La variable objetivo que clasifica la actividad de los usuarios como potenciales clientes es denominada `outcome`, evaluaremos la distribución de este variable: 

In [11]:
print("Distribución de outcome: \n", 100*consolidado["outcome"].value_counts()/consolidado.shape[0])

Distribución de outcome: 
 0    55.60456
1    44.39544
Name: outcome, dtype: float64


Como lo hemos mencionado, las redes neuronales requieren de entrada sólo **valores numéricos**, por lo cual debemos convertir aquellas que no lo son. El dataset consolidado cuenta con 46 variables de tipo booleanas, numericas o categóricas que deben ser transformadas.

In [12]:
types = pd.DataFrame(consolidado.dtypes)
print("Tipos de variables",types.groupby(0).size())

Tipos de variables 0
bool      28
int64      2
object    16
dtype: int64


### Variables booleanas a variables enteras

In [13]:
# reemplezar valores 'False' por 0, y 'True' por 1
consolidado = consolidado.replace({False: 0, True: 1})

# reconteo de tipos de variables
types = pd.DataFrame(consolidado.dtypes)
print("Tipos de variables luego de reemplazo",types.groupby(0).size())

Tipos de variables luego de reemplazo 0
int64     30
object    16
dtype: int64


### Variables object (string) a variables enteras

Si recordamos, el identificador `people_id` se compone por un prefijo "`ppl_`" seguido de un número único por usuario. En este caso basta con recortar el prefijo para transformar esta variable en numerica.

In [14]:
consolidado.people_id = consolidado.people_id.str.slice(start=4).astype(float).astype(int)

types = pd.DataFrame(consolidado.dtypes)
print("Tipos de variables luego de 2do reemplazo",types.groupby(0).size())

Tipos de variables luego de 2do reemplazo 0
int64     31
object    15
dtype: int64


Al validar el dataset consolidado podemos aplicar la misma metodología previamente mencionada a otro par de columnas que tienen las mismas características: 

- `activity_id,`
- `activity_category,`
- `group_1, `
- `tipo_actividad.`

In [15]:
consolidado.head(3)

,people_id,activity_id,fecha_actividad,activity_category,tipo_actividad,outcome,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,char_11,char_12,char_13,char_14,char_15,char_16,char_17,char_18,char_19,char_20,char_21,char_22,char_23,char_24,char_25,char_26,char_27,char_28,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,100,act2_1734928,2023-08-26,type 4,type 76,0,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,type 2,type 2,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,1,1,0,0,1,1,1,0,36
1,100,act2_2434093,2022-09-27,type 2,type 1,0,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,type 2,type 2,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,1,1,0,0,1,1,1,0,36
2,100,act2_3404049,2022-09-27,type 2,type 1,0,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,type 2,type 2,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,1,1,0,0,1,1,1,0,36


In [16]:
consolidado[["activity_id", "activity_category", "group_1", "tipo_actividad"]].head(6)

,activity_id,activity_category,group_1,tipo_actividad
0,act2_1734928,type 4,group 17304,type 76
1,act2_2434093,type 2,group 17304,type 1
2,act2_3404049,type 2,group 17304,type 1
3,act2_3651215,type 2,group 17304,type 1
4,act2_4109017,type 2,group 17304,type 1
5,act2_898576,type 4,group 17304,type 1727


In [17]:
consolidado.activity_id = consolidado.activity_id.str.slice(start=5).astype(float).astype(int)
consolidado.activity_category = consolidado.activity_category.str.slice(start=5).astype(float).astype(int)
consolidado.group_1 = consolidado.group_1.str.slice(start=6).astype(float).astype(int)
consolidado.tipo_actividad = consolidado.tipo_actividad.str.slice(start=5).astype(float).astype(int)

types = pd.DataFrame(consolidado.dtypes)
print("Tipos de variables luego de 3er reemplazo",types.groupby(0).size())

Tipos de variables luego de 3er reemplazo 0
int64     35
object    11
dtype: int64


Evaluemos las variables que siguen tipo `object` y la cantidad de valores distintos que poseen

In [18]:
types[types.values == 'object'].index

Index(['fecha_actividad', 'char_1', 'char_2', 'date', 'char_3', 'char_4',
       'char_5', 'char_6', 'char_7', 'char_8', 'char_9'],
      dtype='object')

In [19]:
# variables de tipo object
objects_list = list(types[types.values == 'object'].index)
objects_list

['fecha_actividad',
 'char_1',
 'char_2',
 'date',
 'char_3',
 'char_4',
 'char_5',
 'char_6',
 'char_7',
 'char_8',
 'char_9']

In [20]:
# cantidad de valores distintos que poseen 
consolidado[objects_list].nunique()

fecha_actividad     411
char_1                2
char_2                3
date               1196
char_3               43
char_4               25
char_5                9
char_6                7
char_7               25
char_8                8
char_9                9
dtype: int64

### Variables de tipo fecha

Del listado anterior resaltamos dos variables tipo fecha (`date`, `fecha_actividad`). Para la red neuronal representa una oportunidad poder extraer una serie de caracteristicas adicionales que le permitan entender patrones estacionales de diferentes periodicidades (diario, semanal, mensual, etc) es por esto que crearemos una serie de variables adicionales (de naturaleza numérica) a partir de estas dos fechas:

In [21]:
#Convertimos la variable objeto en datetime
consolidado["date"] = pd.to_datetime(consolidado["date"])
#Creamos nuevas variables
consolidado["dia"] = consolidado["date"].dt.day
consolidado["dia_semana"] = consolidado["date"].dt.weekday
consolidado["semana"] = consolidado["date"].dt.week
consolidado["mes"] = consolidado["date"].dt.month
consolidado["trimestre"] = consolidado["date"].dt.quarter
consolidado["año"] = consolidado["date"].dt.year

#Repetimos el procedimiento anterior con fecha_actividad
consolidado["fecha_actividad"] = pd.to_datetime(consolidado["fecha_actividad"])
consolidado["dia_actividad"] = consolidado["fecha_actividad"].dt.day
consolidado["dia_semana_actividad"] = consolidado["fecha_actividad"].dt.weekday
consolidado["semana_actividad"] = consolidado["fecha_actividad"].dt.week
consolidado["mes_actividad"] = consolidado["fecha_actividad"].dt.month
consolidado["trimestre_actividad"] = consolidado["fecha_actividad"].dt.quarter
consolidado["año_actividad"] = consolidado["fecha_actividad"].dt.year

#Eliminamos las columnas originales
del(consolidado["date"])
del(consolidado["fecha_actividad"])

types = pd.DataFrame(consolidado.dtypes)
print("Tipos de variables luego de 4to reemplazo",types.groupby(0).size())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  from ipykernel import kernelapp as app


Tipos de variables luego de 4to reemplazo 0
int64     47
object     9
dtype: int64


## One hot encoding

Con la relativa pequeña cantidad de variables categoricas que manejamos vamos a proceder a realizar la conversión por One Hot Encoding

In [22]:
mi_consolidado = consolidado.copy(deep=True)
print(mi_consolidado.shape)

(2197291, 56)


In [23]:
#Se define función que captura el dataframe y la columna para retornar un dataframe luego de OHE
def OneHotEncoding_df(df, columna):
  OHE_df = pd.get_dummies(columna+'_'+df[columna])
  return OHE_df

In [24]:
#Se obtiene las columnas a las que se va a realizar OHE
objects_list = list(types[types.values == 'object'].index)
objects_list

['char_1',
 'char_2',
 'char_3',
 'char_4',
 'char_5',
 'char_6',
 'char_7',
 'char_8',
 'char_9']

In [25]:
#Se realiza OHE para cada categoría
for category in objects_list:
  mask = OneHotEncoding_df(mi_consolidado, category)
  print(f'column {category} transformed!')
  mi_consolidado.drop(category, axis=1, inplace=True)
  mi_consolidado = pd.concat([mi_consolidado, mask], axis=1)

print(f"Tamaño final del dataset transformado: {mi_consolidado.shape}")

column char_1 transformed!
column char_2 transformed!
column char_3 transformed!
column char_4 transformed!
column char_5 transformed!
column char_6 transformed!
column char_7 transformed!
column char_8 transformed!
column char_9 transformed!
Tamaño final del dataset transformado: (2197291, 178)


La columna extra es la variable objetivo: '`outcome`'

In [ ]:
mi_consolidado['outcome']

0          0
1          0
2          0
3          0
4          0
          ..
2197286    1
2197287    1
2197288    1
2197289    1
2197290    1
Name: outcome, Length: 2197291, dtype: int64

# Exportar dataframe

Debido a que se tiene un dataset con una gran cantidad de datos (2197291 muestras y 178 variables), la extensión en la que se exportará los datos será `parquet`, ya que presenta mayor ventaja que un arcivo CSV [🔗](https://www.notion.so/mariajosemv/Manipulaci-n-de-datos-con-Pandas-y-Python-695eff3f1ad343cfab5aa03f9b040690#fe8c31bdf23c41a390d7b6674dac0a2e)

In [28]:
mi_consolidado.to_parquet('./datasets/variables_consolidadas.parquet')